### Python import

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from module.image import canny, get_boundary_points
from module.label_map import get_label_map
from module.config import config_from_yaml
from tqdm import tqdm
tqdm.pandas()

from ISR.models import RDN
rdn = RDN(weights='noise-cancel')

### Initialize

In [ ]:
CFG = config_from_yaml('config.yaml')

### Make train.csv

In [ ]:
def apply_boundary_points(path, x_center, y_center, width, height, label):
    if label == 'Pass':
        return ''
    else:
        img = cv2.imread(path)
        img = rdn.predict(img)
        img = cv2.resize(img, dsize=(100, 100), interpolation=cv2.INTER_AREA)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_size = img.shape[0]

        x1   = np.clip(x_center - width  / 2, 0, img_size)
        y1   = np.clip(y_center - height / 2, 0, img_size)
        x2   = np.clip(x_center + width  / 2, 0, img_size)
        y2   = np.clip(y_center + height / 2, 0, img_size)
        bbox = [x1, y1, x2, y2]

        margin = 4
        box_mask_x1 = np.clip(int(np.round(x1)-margin),   0, 100)
        box_mask_x2 = np.clip(int(np.round(x2)+margin)+1, 0, 100)
        box_mask_y1 = np.clip(int(np.round(y1)-margin),   0, 100)
        box_mask_y2 = np.clip(int(np.round(y2)+margin)+1, 0, 100)

        box_mask = np.zeros((img_size, img_size)).astype(np.uint8)
        box_mask[box_mask_y1:box_mask_y2, box_mask_x1:box_mask_x2] = 1

        mask = canny(img)
        mask = mask * box_mask
        
        if label == 'Bright':
            boundary_points = get_boundary_points(mask, bbox, return_bbox=True)
        else:
            boundary_points = get_boundary_points(mask, bbox, return_bbox=False)
            
        boundary_points = [str(item) for item in boundary_points]
        boundary_points = ' '.join(boundary_points)
        return boundary_points

In [ ]:
df                    = pd.read_csv(CFG.DATA.TRAIN_DATA_PATH)
group                 = df.groupby('img_name')
img_names             = np.sort(df['img_name'].unique())
error_images          = pd.read_csv('error.csv')['error_id'].values
df                    = pd.concat([group.get_group(img_name) for img_name in img_names if img_name not in error_images]).reset_index(drop=True)
df['label']           = df['defect'].apply(lambda x: get_label_map()[x])
df['path']            = '../data/train/' + df['label'] + '/' + df['img_name']
df['boundary_points'] = df.progress_apply(lambda x: apply_boundary_points(x['path'], x['x_center'], x['y_center'], x['width'], x['height'], x['label']), axis=1)
df.to_csv(CFG.DATA.TRAIN_FILE, index=False, encoding='utf-8-sig')